# Retriever, ranker, question answering

The objective of this notebook is to build a simple pipeline to perform the extractive question answering task. Here we build a pipeline with a retriever, a ranker, and a question-answering model. The retriever acts as the first filter in this architecture. The ranker filters the documents a second time based on the semantic similarity between the documents and the question. Finally, we will use a question answering model to extract the answer to the question using filtered documents. 

The model of extractive question answering is slow, even when filtering documents. Therefore, it would make sense to use a GPU for this type of pipeline using QA models.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from cherche import data, rank, retrieve, qa
from sentence_transformers import SentenceTransformer
from transformers import pipeline

We can use the `towns` corpus for this example: we can ask questions about the cities of Bordeaux, Toulouse, Paris and Lyon.

In [3]:
documents = data.load_towns()
documents[:4]

[{'id': 0,
  'title': 'Paris',
  'url': 'https://en.wikipedia.org/wiki/Paris',
  'article': 'Paris (French pronunciation: \u200b[paʁi] (listen)) is the capital and most populous city of France, with an estimated population of 2,175,601 residents as of 2018, in an area of more than 105 square kilometres (41 square miles).'},
 {'id': 1,
  'title': 'Paris',
  'url': 'https://en.wikipedia.org/wiki/Paris',
  'article': "Since the 17th century, Paris has been one of Europe's major centres of finance, diplomacy, commerce, fashion, gastronomy, science, and arts."},
 {'id': 2,
  'title': 'Paris',
  'url': 'https://en.wikipedia.org/wiki/Paris',
  'article': 'The City of Paris is the centre and seat of government of the region and province of Île-de-France, or Paris Region, which has an estimated population of 12,174,880, or about 18 percent of the population of France as of 2017.'},
 {'id': 3,
  'title': 'Paris',
  'url': 'https://en.wikipedia.org/wiki/Paris',
  'article': 'The Paris Region had 

We start by creating a retriever whose mission will be to quickly filter the documents. This retriever will find documents based on the title and content of the article using `on` parameter.

In [4]:
retriever = retrieve.TfIdf(key="id", on=["title", "article"], documents=documents, k = 30)

We then add a ranker to the pipeline to filter the results according to the semantic similarity between the query and the retrieved documents. 
similarity between the query and the retriever's output documents. The ranker will be based on the content of the article.

In [5]:
ranker = rank.Encoder(
    key = "id",
    on = ["title", "article"],
    encoder = SentenceTransformer("sentence-transformers/all-mpnet-base-v2").encode,
    k = 3,
)

In [6]:
question_answering = qa.QA(
    model = pipeline("question-answering", 
        model = "deepset/roberta-base-squad2", 
        tokenizer = "deepset/roberta-base-squad2"
    ),
    on = "article",
)

We initialise the pipeline and ask the retrievers to index the documents and the ranker to pre-compute the document embeddings. This step can take some time if you have a lot of documents. A GPU could speed up the process. Embeddings of the ranker will be stored in the file `encoder.pkl`. Also the question answering model needs the documents fields and not only ids. To map ids to documents, we add the `documents` to our pipeline. 

In [7]:
search = retriever + ranker + documents + question_answering
search.add(documents)

Ranker embeddings calculation.: 100%|█| 2/2 [00:02<00:


TfIdf retriever
 	 key: id
 	 on: title, article
 	 documents: 105
Encoder ranker
	 key: id
	 on: title, article
	 k: 3
	 similarity: cosine
	 Embeddings pre-computed: 105
Mapping to documents
Question Answering
	 on: article

Paris Saint Germain is the name of the biggest football team of Paris. The Question Answering Pipeline provides the ranking-related similarity score called `similarity` and the question answering task-related score `qa_score`. The higher the `qa_score` the more likely the answer is. The answers are sorted from the most likely to the least likely.

In [8]:
search("What is the name of the football club of Paris?")

[{'start': 18,
  'end': 37,
  'answer': 'Paris Saint-Germain',
  'qa_score': 0.9848365783691406,
  'id': 20,
  'title': 'Paris',
  'url': 'https://en.wikipedia.org/wiki/Paris',
  'article': 'The football club Paris Saint-Germain and the rugby union club Stade Français are based in Paris.',
  'similarity': 0.7104821801185608},
 {'start': 16,
  'end': 31,
  'answer': 'Stade de France',
  'qa_score': 0.8121969103813171,
  'id': 21,
  'title': 'Paris',
  'url': 'https://en.wikipedia.org/wiki/Paris',
  'article': 'The 80,000-seat Stade de France, built for the 1998 FIFA World Cup, is located just north of Paris in the neighbouring commune of Saint-Denis.',
  'similarity': 0.4616149663925171},
 {'start': 15,
  'end': 17,
  'answer': '12',
  'qa_score': 0.015906406566500664,
  'id': 16,
  'title': 'Paris',
  'url': 'https://en.wikipedia.org/wiki/Paris',
  'article': 'Paris received 12.',
  'similarity': 0.467741459608078}]

Toulouse in France is known as "The Pink City".

In [9]:
search("What is the color of Toulouse?")

[{'start': 39,
  'end': 46,
  'answer': 'pinkish',
  'qa_score': 0.5257010459899902,
  'id': 40,
  'title': 'Toulouse',
  'url': 'https://en.wikipedia.org/wiki/Toulouse',
  'article': 'The city\'s unique architecture made of pinkish terracotta bricks has earned Toulouse the nickname La Ville Rose ("The Pink City").',
  'similarity': 0.6403290033340454},
 {'start': 11,
  'end': 35,
  'answer': 'too-LOOZ, French: [tuluz',
  'qa_score': 0.024280602112412453,
  'id': 26,
  'title': 'Toulouse',
  'url': 'https://en.wikipedia.org/wiki/Toulouse',
  'article': 'Toulouse ( too-LOOZ, French: [tuluz] (listen); Occitan: Tolosa [tuˈluzɔ]; Latin: Tolosa [tɔˈloːsa]) is the prefecture of the French department of Haute-Garonne and of the larger region of Occitanie.',
  'similarity': 0.5757554173469543},
 {'start': 29,
  'end': 38,
  'answer': 'Occitanie',
  'qa_score': 0.0001560609816806391,
  'id': 37,
  'title': 'Toulouse',
  'url': 'https://en.wikipedia.org/wiki/Toulouse',
  'article': 'It is now th

Bordeaux is known worldwide for its wine.

In [10]:
search("What is the speciality of Bordeaux ?")

[{'start': 31,
  'end': 35,
  'answer': 'wine',
  'qa_score': 0.7739061713218689,
  'id': 65,
  'title': 'Bordeaux',
  'url': 'https://en.wikipedia.org/wiki/Bordeaux',
  'article': "Bordeaux is a world capital of wine, with its castles and vineyards of the Bordeaux region that stand on the hillsides of the Gironde and is home to the world's main wine fair, Vinexpo.",
  'similarity': 0.6415764689445496},
 {'start': 39,
  'end': 49,
  'answer': 'gastronomy',
  'qa_score': 0.43909284472465515,
  'id': 66,
  'title': 'Bordeaux',
  'url': 'https://en.wikipedia.org/wiki/Bordeaux',
  'article': 'Bordeaux is also one of the centers of gastronomy and business tourism for the organization of international congresses.',
  'similarity': 0.6372377872467041},
 {'start': 85,
  'end': 89,
  'answer': 'port',
  'qa_score': 0.3996892273426056,
  'id': 57,
  'title': 'Bordeaux',
  'url': 'https://en.wikipedia.org/wiki/Bordeaux',
  'article': 'Bordeaux ( bor-DOH, French: [bɔʁdo] (listen); Gascon Occitan: 

Every year there is a silk festival in Lyon.

In [11]:
search("What is the speciality of Lyon ?")

[{'start': 41,
  'end': 48,
  'answer': 'banking',
  'qa_score': 0.6367450952529907,
  'id': 52,
  'title': 'Lyon',
  'url': 'https://en.wikipedia.org/wiki/Lyon',
  'article': 'Economically, Lyon is a major centre for banking, as well as for the chemical, pharmaceutical and biotech industries.',
  'similarity': 0.6994298696517944},
 {'start': 74,
  'end': 78,
  'answer': 'silk',
  'qa_score': 0.34435904026031494,
  'id': 49,
  'title': 'Lyon',
  'url': 'https://en.wikipedia.org/wiki/Lyon',
  'article': 'Lyon was historically an important area for the production and weaving of silk.',
  'similarity': 0.6723434329032898},
 {'start': 20,
  'end': 28,
  'answer': 'economic',
  'qa_score': 0.16321411728858948,
  'id': 47,
  'title': 'Lyon',
  'url': 'https://en.wikipedia.org/wiki/Lyon',
  'article': 'Lyon became a major economic hub during the Renaissance.',
  'similarity': 0.6418805718421936}]

The retriever ranker pipeline is fast enough on cpu but the question answering model takes time.

In [12]:
%timeit (retriever + ranker)("What is the speciality of Lyon ?")

43.9 ms ± 3.25 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
%timeit (retriever + ranker + documents + question_answering)("What is the speciality of Lyon ?")

142 ms ± 2.25 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
